In [3]:
import numpy as np
import tensorflow as tf
import torch
from tqdm.keras import TqdmCallback
from eikonalfm import factored_fast_marching as ffm
from eikonalfm import distance
from EikoNet import model as EikoNet
import holoviews as hv
hv.extension('matplotlib')
from IPython.display import clear_output
import misc
import velocity

clear_output()

# Loading velocity model

In [4]:
# EikoNet by default is applied for 3D models, create pseudo 3D model of Marmousi
Vel2D = misc.Marmousi(smooth=3, section=[[600, 881], None])
vmin, vmax = Vel2D.min, Vel2D.max
xmin, zmin = Vel2D.xmin
xmax, zmax = Vel2D.xmax
nz, nx = 281, 281

x = np.linspace(xmin, xmax, nx)
z = np.linspace(zmin, zmax, nz)
Xr_2d = np.stack(np.meshgrid(x, z, indexing='ij'), axis=-1)
V_2d = Vel2D(Xr_2d)

# Converting to pseudo 3D
y = np.array([0.0, 1e-3])
V_3d = np.tile(V_2d[..., None], reps=(1, 1, len(y)))

Xr_3d = np.stack(np.meshgrid(x, z, y, indexing='ij'), axis=-1)
Vel3D = velocity.Interpolator(V_3d, x, z, y)


In [5]:
# Adaption for EikoNet format

class EikoNetVelocity:
    def __init__(self, Vel):
        self.xmin = Vel.xmin
        self.xmax = Vel.xmax
        self.projection = None
        self.f = Vel
    
    def eval(self, Xp):
        Yp = np.zeros((Xp.shape[0], 2))
        Yp[:, 0] = self.f(Xp[:, :3])
        Yp[:, 1] = self.f(Xp[:, 3:])
        return Yp

EikoNet_Vel3D = EikoNetVelocity(Vel3D)

In [6]:
# Test source location
Ixs = [
    (nx//9, nz//9, 0),
    (nx//2, nz//2, 0),
    (int(nx//1.1), int(nz//1.1), 0)
]
d = [x[1] - x[0], z[1] - z[0], y[1] - y[0]]
T_ref = []

for ixs in Ixs:
    D = distance(V_3d.shape, d, ixs, indexing='ij')[..., 0]
    T_ref.append(D * ffm(V_3d, ixs, d, 2)[..., 0])

T_ref = np.stack(T_ref, axis=0)

In [7]:
xr = Xr_3d[..., 0, :]
Xs = [np.tile(Xr_3d[ixs][None, None, ...], xr.shape[:-1] + (1,)) for ixs in Ixs]
Xp = [np.concatenate([xsi, xr], axis=-1) for xsi in Xs]
X_test = np.stack(Xp, axis=0)

# Training EikoNet

In [9]:
filePath = './data'
model = EikoNet.Model(filePath, VelocityClass=EikoNet_Vel3D, device='cuda:0')
model.Params['Training']['Number of sample points'] = 5000
model.Params['Training']['Save Every * Epoch'] = 1e6
model.Params['Training']['Print Every * Epoch'] = 100
model.Params['Training']['Number of Epochs'] = 1000

In [7]:
%%time
model.train()
EikoNet_loss = model.total_train_loss

(5000, 6) (5000, 2)
cpu
========================== Eikonal Solver - Training ==================================
Epoch = 100 -- Training loss = 1.0513e-01 -- Validation loss = 6.1825e-02
Epoch = 200 -- Training loss = 1.0499e-01 -- Validation loss = 6.1837e-02
Epoch = 300 -- Training loss = 1.0595e-01 -- Validation loss = 6.1837e-02
Epoch = 400 -- Training loss = 1.0267e-01 -- Validation loss = 6.1858e-02
Epoch = 500 -- Training loss = 1.0697e-01 -- Validation loss = 6.1831e-02
Epoch = 600 -- Training loss = 1.0418e-01 -- Validation loss = 6.1804e-02
Epoch = 700 -- Training loss = 1.0333e-01 -- Validation loss = 6.1778e-02
Epoch = 800 -- Training loss = 1.0654e-01 -- Validation loss = 6.1748e-02
Epoch = 900 -- Training loss = 1.0425e-01 -- Validation loss = 6.1790e-02
Epoch = 1000 -- Training loss = 1.0381e-01 -- Validation loss = 6.1646e-02
CPU times: user 7h 56min 24s, sys: 38.4 s, total: 7h 57min 2s
Wall time: 47min 56s


In [10]:
# After training EikoNet, GPU memory is not cleaned
# To run it for prediction, we load saved model on CPU to use RAM

model2 = EikoNet.Model(filePath, EikoNet_Vel3D, device='cpu')

saved_model_name = 'Model_Epoch_01000_ValLoss_0.8574802279472351.pt'
model2.load(f"{filePath}/{saved_model_name}")

Xpt = torch.tensor(np.float32(X_test.reshape(-1, 6)))
TT = model2.TravelTimes(Xpt)
T_eikonet = TT.detach().numpy().reshape(len(Ixs), nx, nz)

/home/andrei/.local/lib/python3.11/site-packages/EikoNet/model.py:343: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint            = torch.load(filepath,map_location

# Visualization

In [1]:
figs = []
labels = ['f-FMM', 'EikoNet']
colors = ['w', 'b']
linestyle = ['solid', 'dashed']
linewidth = [1, 2]

for t_ref, t_eikonet in zip(T_ref, T_eikonet):
    ixs = np.unravel_index(t_ref.argmin(), t_ref.shape)
    vmap = hv.Image((x, z, V_2d.T), vdims='Velocity (km/s)', kdims=['X (km)', 'Z (km)']).opts(cmap='viridis', colorbar=True)

solutions = [t_ref, t_eikonet]
levels = np.linspace(0, np.nanmax(t_ref), 11)

tmaps = [hv.Image((x, z, solutions[i].T), vdims='Time (s)', label=labels[i]) for i in range(len(labels))]
contours = [hv.operation.contours(tmaps[i], levels=levels).opt(color=colors[i], cmap=[colors[i]], linestyle=linestyle[i], linewidth=linewidth[i]) for i in range(len(tmaps))]

source_point = hv.Scatter((x[ixs[0]], z[ixs[1]])).opts(marker='*', s=200, c='r')
fig = hv.Overlay([vmap] + contours + [source_point])
fig = fig.opts(hv.opts.Image(fig_size=150, aspect=xmax/zmax, invert_yaxis=True, fontscale=dict(ticks=16, labels=16, legend=16)))
figs.append(fig.opts(show_legend=False))

fig[0] = fig[0].opts(hv.opts.Image(colorbar=False)).opts(yaxis='left')
fig[1] = fig[1].opts(hv.opts.Image(colorbar=False)).opts(show_legend=True, yaxis='bare', legend_opts=dict(loc=(-0.18, 1.05), ncol=3, framealpha=0.5), fontsize=dict(ticks=16, labels=16, legend=18))
fig[2] = fig[2].opts(hv.opts.Image(yaxis='bare'), hv.opts.Scatter(yaxis='bare'))

fig = hv.Layout(figs).cols(3).opts(hspace=0.1, fig_size=125, aspect_weight=1, sublabel_format=' ')

fig

NameError: name 'T_ref' is not defined